In [1]:
# Step 1: 환경 변수 및 패키지 로드
import os
import pickle
import json
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection
from dotenv import load_dotenv
import openai
from openai.types import CreateEmbeddingResponse, Embedding


def load_environment_variables():
    load_dotenv()
    openai.api_key = os.getenv("OPEN_API_KEY")

load_environment_variables()


In [2]:
# Step 2: Milvus 연결 및 데이터 로드
def connect_to_milvus():
    connections.connect(alias="default", host="localhost", port="19530")

def load_data(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

connect_to_milvus()
raw_data = load_data('../final_result.pkl')


In [3]:
# Step 3: 데이터 처리 및 저장
def process_data(raw_data):
    return [{"question": k, "answer": v} for k, v in raw_data.items()]

def save_to_json(data, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

faqs = process_data(raw_data)
save_to_json(faqs, 'faqs.json')

print(f"Sample FAQ: {faqs[0]}")


Sample FAQ: {'question': '[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기)', 'answer': '네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 편리하게 이용하실 수 있습니다.네이버 커머스 ID가 없으시다면 [가입하기] 버튼을 통해 회원가입해 주세요.\xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa01. \'네이버ID\' 로 네이버 커머스 ID 가입하기1) [네이버 아이디로 가입하기] 버튼을 눌러서 네이버 로그인 해 주세요.2) 연락가능한 휴대폰번호를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.\xa03) 네이버 커머스 ID 회원가입이 완료되어 해당 네이버ID 로 스마트스토어센터 서비스 가입을 하실 수 있습니다.\xa0이 후 스마트스토어센터 로그인 시에 [네이버 아이디로 로그인] 버튼을 눌러서 네이버 로그인을 하실 수 있습니다.\xa02. \'이메일 아이디\'로 네이버 커머스 ID 가입하기\xa01) [이메일 아이디로 가입하기] 버튼을 눌러서 사용할 ID를 입력해 주세요.! 중요. 로그인 ID는 "실 사용중인 이메일주소"로 기재하셔야 합니다. (예 : abc@naver.com)2) 연락가능한 휴대폰번호와 이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.3) 네이버 커머스 ID 회원가입이 완료되어 해당 이메일 아이디로 스마트스토어센터 서비스 가입을 하실 수 있습니다.이 후 스마트스토어센터 로그인 시에 해당 이메일 아이디와 비밀번호를 입력하여 로그인을 하실 수 있습니다.\xa0※ 참고. 로그인 ID 계정 대표자와 사업자번호 대표자 동일여부는 체크하지 않습니다. (대표자가 상이해도 가입 진행 가능합니다.)\xa0 \xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa0\n\n\n위 도움말이 도움이 되었나요?\n\n\n별점1점\n\n별점2점\n\n별점3점\n\n별점4점\n\n별점5점\n\n\n\n소중한

In [17]:
# Step 4: Milvus 컬렉션 정의 및 생성
def define_collection_schema():
    fields = [
        FieldSchema(name="faq_id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1536),
        FieldSchema(name="faq", dtype=DataType.JSON),
    ]
    return CollectionSchema(fields, description="FAQ collection")

def create_collection(schema):
    collection = Collection("faq_collection", schema)
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "IP",
        "params": {"nlist": 128}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

schema = define_collection_schema()
collection = create_collection(schema)


In [12]:
# Step 5: 임베딩 생성 및 데이터 삽입 (수정)
def generate_embeddings(questions):
    embeddings = []
    for question in questions:
        response: CreateEmbeddingResponse = openai.embeddings.create(
            model="text-embedding-ada-002",
            input=question
        )
        print(response)
        embedding = response.data
        embeddings.append(embedding)
    return embeddings

# 질문 및 FAQ 데이터 제한
questions = [faq["question"] for faq in faqs][:1]
faqs_limited = faqs[:1]

# 임베딩 생성
embeddings = generate_embeddings(questions)




RPC error: [insert_rows], <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, {embedding} field should be a float_vector, but got a {<class 'list'>} instead.)>, <Time:{'RPC start': '2024-12-06 21:45:42.809207', 'RPC error': '2024-12-06 21:45:42.809640'}>


CreateEmbeddingResponse(data=[Embedding(embedding=[0.012372679077088833, -0.0006586262607015669, 0.025401972234249115, -0.04781699180603027, -0.040117863565683365, 0.016711484640836716, -0.021114666014909744, -0.0032991664484143257, 0.0018298303475603461, 0.001112865167669952, 0.014934763312339783, -0.0012206915998831391, -0.016106370836496353, -0.019505316391587257, 0.00402337359264493, 0.020638298243284225, 0.007660503499209881, -0.005967468023300171, 0.025543594732880592, -0.011941373348236084, -0.007525318302214146, 0.006553271319717169, -0.004834486171603203, -0.012746048159897327, 0.009405038319528103, -0.004638145212084055, 0.014226649887859821, -0.01741959899663925, 0.0025685217697173357, 0.0009607816464267671, 0.012314742431044579, -0.004100622609257698, -0.01165812835097313, -0.015372506342828274, -0.011304071173071861, 0.02460373565554619, -0.013724532909691334, -0.009900717996060848, 0.009334227070212364, -0.03437570482492447, 0.008754861541092396, 0.008059622719883919, 0.0

DataNotMatchException: <DataNotMatchException: (code=1, message=The Input data type is inconsistent with defined schema, {embedding} field should be a float_vector, but got a {<class 'list'>} instead.)>

In [24]:

entities = [
    {
        "embedding": embedding[0].embedding,
        "faq": faq
    } for embedding, faq in zip(embeddings, faqs_limited)
]

# 데이터 삽입
collection.insert(entities)

(insert count: 1, delete count: 0, upsert count: 0, timestamp: 454423926647554050, success count: 1, err count: 0

In [25]:
# Step 6: 컬렉션 정보 출력
def print_collection_info(collection):
    print("컬렉션 필드 정보:")
    for field in collection.schema.fields:
        print(f"필드 이름: {field.name}, 데이터 타입: {field.dtype}")

print_collection_info(collection)


컬렉션 필드 정보:
필드 이름: faq_id, 데이터 타입: 5
필드 이름: embedding, 데이터 타입: 101
필드 이름: faq, 데이터 타입: 23


In [26]:
# Step 7: 컬렉션 쿼리 및 결과 출력
def query_collection(collection):
    collection.load()
    search_params = {"metric_type": "IP", "params": {"nprobe": 10}}
    query_vector = collection.query(
        expr="faq_id >= 0",
        output_fields=["embedding", "faq"],
        limit=5
    )
    return query_vector

def print_query_results(query_vector):
    for result in query_vector:
        print(f"임베딩 데이터: {result['embedding']}")
        print(f"FAQ 데이터: {result['faq']}")

query_vector = query_collection(collection)
print_query_results(query_vector)


임베딩 데이터: [np.float32(0.012372679), np.float32(-0.00065862626), np.float32(0.025401972), np.float32(-0.04781699), np.float32(-0.040117864), np.float32(0.016711485), np.float32(-0.021114666), np.float32(-0.0032991664), np.float32(0.0018298303), np.float32(0.0011128652), np.float32(0.014934763), np.float32(-0.0012206916), np.float32(-0.01610637), np.float32(-0.019505316), np.float32(0.0040233736), np.float32(0.020638298), np.float32(0.0076605035), np.float32(-0.005967468), np.float32(0.025543595), np.float32(-0.011941373), np.float32(-0.0075253183), np.float32(0.0065532713), np.float32(-0.004834486), np.float32(-0.012746048), np.float32(0.009405038), np.float32(-0.004638145), np.float32(0.01422665), np.float32(-0.017419599), np.float32(0.0025685218), np.float32(0.00096078165), np.float32(0.012314742), np.float32(-0.0041006226), np.float32(-0.011658128), np.float32(-0.015372506), np.float32(-0.011304071), np.float32(0.024603736), np.float32(-0.013724533), np.float32(-0.009900718), np.float

In [31]:


# Step 7: 컬렉션 쿼리 및 결과 출력
q1 = "스마트스토어센터 회원가입은 어떻게 하나요? "
data = generate_embeddings([q1])[0][0].embedding


CreateEmbeddingResponse(data=[Embedding(embedding=[0.011613709852099419, -0.013305646367371082, 0.007240212522447109, -0.05914755538105965, -0.0357796736061573, 0.018809229135513306, -0.00962807796895504, -0.013637648895382881, -0.0035434903111308813, -0.00011522329441504553, -0.0014349539997056127, 0.00791698694229126, -0.022078178822994232, -0.022193104028701782, 0.008402221836149693, 0.024427736178040504, 0.012169175781309605, -0.012954489327967167, 0.019115693867206573, -0.02770945616066456, 0.003168391063809395, -0.0025043857749551535, -0.01104547455906868, -0.005276927258819342, 0.017570603638887405, 0.0009082189062610269, 0.02119709551334381, -0.013841958716511726, 0.00885553378611803, -0.0035339132882654667, 0.017800452187657356, -0.0009648828417994082, -0.02451712265610695, -0.008785301819443703, -0.014122883789241314, 0.024644816294312477, -0.006470860447734594, -0.00738067552447319, 0.007923372089862823, -0.022461259737610817, 0.006991210393607616, 0.005372697487473488, 0.01

In [45]:
print(len(data))
search_vector = [data]
for float_data in search_vector:
    print(type(float_data))
print(type(search_vector))  # 데이터가 리스트인지 확인
print(len(search_vector))   # 리스트 길이 확인
search_param = {
    "metric_type": "IP",  # 유사도 측정 방식
    "params": {"nprobe": 10}  # 검색 정확도 조정
}

results = collection.search(
    data=search_vector,
    anns_field="embedding",  
    param=search_param,
    limit=5, 
    output_fields=["faq"]  # 반환할 추가 필드
)

# 결과 출력
for i, result in enumerate(results[0]):
    print(f"Rank {i + 1}: id={result.id}, distance={result.distance}, faq={result.entity.get('faq')}")

1536
<class 'list'>
<class 'list'>
1
Rank 1: id=454422749880753136, distance=0.954073429107666, faq={'question': '[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기)', 'answer': '네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 편리하게 이용하실 수 있습니다.네이버 커머스 ID가 없으시다면 [가입하기] 버튼을 통해 회원가입해 주세요.\xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa01. \'네이버ID\' 로 네이버 커머스 ID 가입하기1) [네이버 아이디로 가입하기] 버튼을 눌러서 네이버 로그인 해 주세요.2) 연락가능한 휴대폰번호를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.\xa03) 네이버 커머스 ID 회원가입이 완료되어 해당 네이버ID 로 스마트스토어센터 서비스 가입을 하실 수 있습니다.\xa0이 후 스마트스토어센터 로그인 시에 [네이버 아이디로 로그인] 버튼을 눌러서 네이버 로그인을 하실 수 있습니다.\xa02. \'이메일 아이디\'로 네이버 커머스 ID 가입하기\xa01) [이메일 아이디로 가입하기] 버튼을 눌러서 사용할 ID를 입력해 주세요.! 중요. 로그인 ID는 "실 사용중인 이메일주소"로 기재하셔야 합니다. (예 : abc@naver.com)2) 연락가능한 휴대폰번호와 이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.3) 네이버 커머스 ID 회원가입이 완료되어 해당 이메일 아이디로 스마트스토어센터 서비스 가입을 하실 수 있습니다.이 후 스마트스토어센터 로그인 시에 해당 이메일 아이디와 비밀번호를 입력하여 로그인을 하실 수 있습니다.\xa0※ 참고. 로그인 ID 계정 대표자와 사업자번호 대표자 동일여부는 체크하지 않습니다. (대표자가 상이해도 가입 진행 가능합니다.)\xa0 \xa0\xa0 \xa0 \xa0 